# 06 - Raven calibration

## Calibration of a Raven model

In this notebook, we show how to calibrate a Raven model using the GR4J-CN predefined structure. The users can refer themselves to the documentation to the parameterization of other hydrological model structures.

In [1]:
import os
from glob import glob
import datetime as dt
from pathlib import Path
from ravenpy.utilities.testdata import get_file, get_local_testdata

In [2]:
from ravenpy.models import GR4JCN, GR4JCN_OST

## Preparing the model to be calibrated on a given watershed
A random watershed is selected for this test. It can be replaced with any desired watershed.

In [3]:
forcing = get_file("raven-gr4j-cemaneige/Salmon-River-Near-Prince-George_meteo_daily.nc")

# Display the datasets that we will be using
display(forcing)

PosixPath('/notebook_dir/writable-workspace/.home/.raven_testing_data/master/raven-gr4j-cemaneige/Salmon-River-Near-Prince-George_meteo_daily.nc')

The selected model will be calibrated using the Ostrich library.

For other model structures (e.g. HMETS, MOHYSE or HBV-EC, please refer to the user manual).

In [4]:
# Using Ostrich with the GR4JCN model
model = GR4JCN_OST()
salmon_land_hru_1 = dict(
    area=4250.6, 
    elevation=843.0, 
    latitude=54.4848, 
    longitude=-123.3659
)

In [5]:
model.config.rvh.hrus=(GR4JCN.LandHRU(**salmon_land_hru_1),)

Ostrich requires a set of initial parameters `params` and its `lower` and `upper` boundaries

In [6]:
params = (0.529, -3.396, 407.29, 1.072, 16.9, 0.053)
lower = (0.01, -15.0, 10.0, 0.0, 1.0, 0.0)
upper = (2.5, 10.0, 700.0, 7.0, 30.0, 1.0)

## Calibration of the selected model
The model can be calibrated by feeding it the following informations:
* forcing: input hydrometeorological data in the right model format
* start_date: starting date of the simulation
* duration: number of days to simulate
* params: initial parameters' values
* lowerBounds: lower boundaries of the parameters
* upperBounds: upper boundaries of the parameters
* algorithm: the optimization algorithm
* random_seed=0,
* max_iterations: maximum number of model iterations performed by the algoritm
* overwrite: overwrite any previous parameter set

In [7]:
# Here, the DDS algorithm with a maximum of 50 model iterations is used.
model(
    forcing,
    start_date=dt.datetime(1980, 1, 1),
    duration=200,
    params=params,
    lowerBounds=lower,
    upperBounds=upper,
    algorithm="DDS",
    random_seed=0,
    max_iterations=50,
    overwrite=True,
)

d = model.diagnostics

/opt/conda/envs/birdy/lib/python3.8/site-packages/ravenpy/models/base.py:434: RavenWarning: Gauge:Initialize: both snowfall and rainfall data are provided at a gauge, but :RainSnowFraction method is something other than RAINSNOW_DATA. Snow fraction will be recalculated.
  warn(msg, category=RavenWarning)
/opt/conda/envs/birdy/lib/python3.8/site-packages/ravenpy/models/base.py:434: RavenWarning: PET data supplied at gauge Gauge1 but will not be used due to choice of forcing generation algorithm
  warn(msg, category=RavenWarning)
/opt/conda/envs/birdy/lib/python3.8/site-packages/ravenpy/models/base.py:434: RavenWarning: CModelInitialize: if a non-standard calendar is used, all observation data must be converted into the same calendar format
  warn(msg, category=RavenWarning)
/opt/conda/envs/birdy/lib/python3.8/site-packages/ravenpy/models/base.py:434: RavenWarning: CModelInitialize: if a non-standard calendar is used, care must be taken with forcing data. All gauge forcing data must use 

## Analysing the calibration results
The best parameter set as well as objective functions can be analyzed.

In [8]:
print('Nash-Sutcliffe value is: ' + str(d['DIAG_NASH_SUTCLIFFE']))
print(model.calibrated_params) # With explanations of what these parameters are
print(model.optimized_parameters) # Just the array that could be used in another process. This is what people will want to use.

Nash-Sutcliffe value is: [0.415253]
GR4JCN.Params(GR4J_X1=1.615284, GR4J_X2=-1.738561, GR4J_X3=119.4733, GR4J_X4=6.883103, CEMANEIGE_X1=14.26573, CEMANEIGE_X2=0.8991888)
[  1.615284   -1.738561  119.4733      6.883103   14.26573     0.8991888]
